In [1]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3 import PPO

# Define a custom traffic control environment
class TrafficControlEnv(gym.Env):
    def __init__(self):
        super(TrafficControlEnv, self).__init__()

        # Define action and observation space
        # Actions: 0 -> Switch to Red, 1 -> Switch to Green, 2 -> Extend Green Light
        self.action_space = spaces.Discrete(3)

        # Observation space: [vehicle count, average waiting time, traffic density, vehicle type priority]
        self.observation_space = spaces.Box(low=0, high=100, shape=(4,), dtype=np.float32)

        # Initialize state variables
        self.vehicle_count = 0
        self.max_vehicles = 100
        self.average_waiting_time = 0
        self.traffic_density = 0
        self.vehicle_type_priority = 0  # Priority given to buses, trucks, etc.

        self.green_light_duration = 10
        self.current_light_state = 1  # 1 for green light, 0 for red light
        self.time_step = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self.vehicle_count = np.random.randint(0, self.max_vehicles)
        self.average_waiting_time = 0
        self.traffic_density = self.vehicle_count / self.max_vehicles
        self.vehicle_type_priority = np.random.uniform(0, 1)  # Random priority for large vehicles

        self.current_light_state = 1
        self.time_step = 0

        return np.array([self.vehicle_count, self.average_waiting_time, self.traffic_density, self.vehicle_type_priority], dtype=np.float32), {}

    def step(self, action):
        self.time_step += 1

        # Update the vehicle count randomly to simulate traffic inflow/outflow
        new_vehicle_count = self.vehicle_count + np.random.randint(-5, 5)
        new_vehicle_count = max(0, min(self.max_vehicles, new_vehicle_count))

        # Update average waiting time (increase when red light, reset when green light)
        if self.current_light_state == 0:
            self.average_waiting_time += 1
        else:
            self.average_waiting_time = 0

        # Update traffic density based on vehicle count
        self.traffic_density = new_vehicle_count / self.max_vehicles

        # Update vehicle type priority (random for now, could be based on actual vehicle data)
        self.vehicle_type_priority = np.random.uniform(0, 1)

        # Handle actions
        if action == 0:  # Switch to Red Light
            self.current_light_state = 0
        elif action == 1:  # Switch to Green Light
            self.current_light_state = 1
        elif action == 2:  # Extend Green Light
            self.current_light_state = 1
            self.green_light_duration += 1

        # Reward is based on reducing vehicle waiting time and density
        reward = -self.average_waiting_time - self.traffic_density * 100

        done = self.time_step >= 100

        # Update state
        self.vehicle_count = new_vehicle_count

        return np.array([self.vehicle_count, self.average_waiting_time, self.traffic_density, self.vehicle_type_priority], dtype=np.float32), reward, done, False, {}

    def render(self, mode="human"):
        print(f"Time Step: {self.time_step}, Vehicle Count: {self.vehicle_count}, Waiting Time: {self.average_waiting_time}, Light: {'Green' if self.current_light_state else 'Red'}")


# Test the environment
env = TrafficControlEnv()

# Reset environment before training
state, _ = env.reset()

# Initialize the PPO model using Stable Baselines3
model = PPO("MlpPolicy", env, verbose=1)

# Train the model for a certain number of timesteps
model.learn(total_timesteps=10000)

# Save the trained model
model.save("traffic_control_agent_v2")

# Test the trained model
state, _ = env.reset()
for _ in range(100):
    action, _states = model.predict(state)
    state, reward, done, truncated, info = env.step(action)
    env.render()

    if done:
        break

# Close the environment
env.close()


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 100       |
|    ep_rew_mean     | -3.96e+03 |
| time/              |           |
|    fps             | 1934      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 2048      |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | -3.44e+03    |
| time/                   |              |
|    fps                  | 1267         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0068800724 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2         

In [3]:
import cv2
import torch
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3 import PPO

# Load YOLOv5 model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Define the path to the video file
video_path = r'C:\Users\kligh\OneDrive\Desktop\Machine_Learning_Projects()\Traffic_Control_YoloV5\trafficnet_dataset_v1\traffic_video_test_1.mp4'

# Define a custom traffic control environment that uses YOLOv5 for vehicle detection
class TrafficControlEnv(gym.Env):
    def __init__(self):
        super(TrafficControlEnv, self).__init__()

        # Define action and observation space
        # Actions: 0 -> Switch to Red, 1 -> Switch to Green, 2 -> Extend Green Light
        self.action_space = spaces.Discrete(3)

        # Observation space: [vehicle count, average waiting time, traffic density, vehicle type priority]
        self.observation_space = spaces.Box(low=0, high=100, shape=(4,), dtype=np.float32)

        # Initialize state variables
        self.vehicle_count = 0
        self.max_vehicles = 100
        self.average_waiting_time = 0
        self.traffic_density = 0
        self.vehicle_type_priority = 0

        self.green_light_duration = 10
        self.current_light_state = 1  # 1 for green light, 0 for red light
        self.time_step = 0
        self.video = cv2.VideoCapture(video_path)  # Open the video file

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.video.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset video to start

        # Reset state variables
        self.vehicle_count = 0
        self.average_waiting_time = 0
        self.traffic_density = 0
        self.vehicle_type_priority = 0

        self.current_light_state = 1  # Start with green light
        self.time_step = 0

        return np.array([self.vehicle_count, self.average_waiting_time, self.traffic_density, self.vehicle_type_priority], dtype=np.float32), {}

    def step(self, action):
        self.time_step += 1

        # Read a frame from the video
        ret, frame = self.video.read()
        if not ret:
            print("End of video.")
            done = True
            return np.array([self.vehicle_count, self.average_waiting_time, self.traffic_density, self.vehicle_type_priority], dtype=np.float32), 0, done, False, {}

        # Use YOLOv5 to detect vehicles in the frame
        results = yolo_model(frame)

        # Filter for vehicle-related objects (car, bus, truck)
        vehicle_classes = ['car', 'bus', 'truck']
        detected_classes = results.xyxy[0][:, -1].cpu().numpy()
        self.vehicle_count = sum([1 for cls in detected_classes if results.names[int(cls)] in vehicle_classes])

        # Update average waiting time
        if self.current_light_state == 0:  # Red light
            self.average_waiting_time += 1
        else:
            self.average_waiting_time = 0

        # Update traffic density based on vehicle count
        self.traffic_density = self.vehicle_count / self.max_vehicles

        # Vehicle type priority (based on types detected, higher priority for buses/trucks)
        self.vehicle_type_priority = np.random.uniform(0, 1)  # Random for now, but could use real priorities

        # Handle actions
        if action == 0:  # Switch to Red Light
            self.current_light_state = 0
        elif action == 1:  # Switch to Green Light
            self.current_light_state = 1
        elif action == 2:  # Extend Green Light
            self.current_light_state = 1
            self.green_light_duration += 1

        # Reward based on waiting time and traffic density
        reward = -self.average_waiting_time - self.traffic_density * 100

        # Done condition if the video ends or max timesteps reached
        done = self.time_step >= 100

        return np.array([self.vehicle_count, self.average_waiting_time, self.traffic_density, self.vehicle_type_priority], dtype=np.float32), reward, done, False, {}

    def render(self, mode="human"):
        print(f"Time Step: {self.time_step}, Vehicle Count: {self.vehicle_count}, Waiting Time: {self.average_waiting_time}, Light: {'Green' if self.current_light_state else 'Red'}")

# Initialize the environment and agent
env = TrafficControlEnv()

# Load the trained model or initialize a new one if you need to train
model = PPO("MlpPolicy", env, verbose=1)

# Train the model for a certain number of timesteps (optional, skip if the model is trained)
# model.learn(total_timesteps=10000)

# Test the trained model on the video
state, _ = env.reset()
while True:
    action, _states = model.predict(state)
    state, reward, done, truncated, info = env.step(action)
    env.render()

    if done:
        break

# Release video resources
env.video.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-19 Python-3.9.18 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Time Step: 1, Vehicle Count: 2, Waiting Time: 0, Light: Green
Time Step: 2, Vehicle Count: 4, Waiting Time: 0, Light: Green
Time Step: 3, Vehicle Count: 2, Waiting Time: 0, Light: Green


C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Time Step: 4, Vehicle Count: 3, Waiting Time: 0, Light: Green
Time Step: 5, Vehicle Count: 3, Waiting Time: 0, Light: Red
Time Step: 6, Vehicle Count: 1, Waiting Time: 1, Light: Green


C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Time Step: 96, Vehicle Count: 0, Waiting Time: 0, Light: Green
Time Step: 97, Vehicle Count: 0, Waiting Time: 0, Light: Red
Time Step: 98, Vehicle Count: 1, Waiting Time: 1, Light: Green


C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Time Step: 99, Vehicle Count: 4, Waiting Time: 0, Light: Green
Time Step: 100, Vehicle Count: 4, Waiting Time: 0, Light: Red


C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [3]:
import gym
from gym import spaces
import numpy as np
import torch
import cv2
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from ultralytics import YOLO  # Assuming you're using YOLOv5 through Ultralytics library


# Define the custom TrafficLightEnv
class TrafficLightEnv(gym.Env):
    def __init__(self):
        super(TrafficLightEnv, self).__init__()
        # Define action and observation space
        self.action_space = spaces.Discrete(2)  # Red or Green light
        self.observation_space = spaces.Box(low=0, high=100, shape=(2,), dtype=np.int32)  # Vehicle count, waiting time
        self.state = None

    def reset(self):
        self.state = np.array([np.random.randint(0, 50), 0])  # Random vehicle count, 0 waiting time
        return self.state

    def step(self, action):
        vehicle_count, waiting_time = self.state
        if action == 1:  # Green light
            waiting_time = 0
            vehicle_count = max(vehicle_count - np.random.randint(5, 20), 0)
        else:  # Red light
            waiting_time += 1

        self.state = np.array([vehicle_count, waiting_time])
        reward = -waiting_time  # Negative reward to minimize waiting time
        done = vehicle_count == 0
        return self.state, reward, done, {}

    def render(self, mode='human'):
        print(f"Vehicle Count: {self.state[0]}, Waiting Time: {self.state[1]}")

# Register custom environment
gym.envs.registration.register(
    id='TrafficLightEnv-v0',
    entry_point='__main__:TrafficLightEnv',
)

# Process the video with YOLOv5 and the RL agent
def process_video_with_rl(video_path):
    # Load YOLOv5 model
    model = YOLO('yolov5s')  # Assuming you have the YOLOv5s model ready

    # Initialize video capture
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_path = "output.mp4"
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

    # Initialize the RL environment (TrafficLightEnv)
    env = gym.make('TrafficLightEnv-v0')
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])

    # Initialize PPO model
    model_rl = PPO('MlpPolicy', env, verbose=1)
    model_rl.learn(total_timesteps=10000)

    time_step = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run YOLOv5 detection
        results = model(frame)

        # Render bounding boxes on frame
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy.cpu().numpy().astype(int)[0]
                label = box.cls
                conf = box.conf
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        # Get RL action (Red or Green light)
        action, _states = model_rl.predict(env.reset())
        new_state, reward, done, info = env.step(action)
        print(f"Time Step: {time_step + 1}, Vehicle Count: {new_state[0][0]}, Waiting Time: {new_state[0][1]}, Light: {'Green' if action == 1 else 'Red'}")

        # Write the processed frame
        out.write(frame)
        time_step += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()


# Call the process_video_with_rl with your video file
video_path = 'traffic_video.mp4'  # Replace with your actual video file path
process_video_with_rl(video_path)


FileNotFoundError: 'yolov5s' does not exist